In [1]:
import re
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib

import pandas as pd

In [2]:
team = pd.read_csv("2020빅콘테스트_스포츠투아이_제공데이터_팀_2020.csv", engine = 'python')

In [3]:
tmp1 = list(team.T_NM)
tmp2 = list(team.T_ID)

team_dict = dict()

for i in range(len(tmp1)):
    team_dict[tmp1[i]] = tmp2[i]
team_dict

{'한화': 'HH',
 'KIA': 'HT',
 'KT': 'KT',
 'LG': 'LG',
 '롯데': 'LT',
 'NC': 'NC',
 '두산': 'OB',
 'SK': 'SK',
 '삼성': 'SS',
 '키움': 'WO'}

### 1. 함수

In [4]:
def read_txt(file_name):

    # 날짜 데이터 읽기
    f = open(file_name, mode = 'r')
    f_urls = f.readlines()
    urls = f_urls[0][1:-1].split(',')
    
    tmp_url = urls[0][1:-1]
    
    return urls, tmp_url

In [10]:
def make_dataframe(tmp_url, file_name):
    fp = urlopen(tmp_url)
    source = fp.read()
    fp.close()

    soup = BeautifulSoup(source)

    table = soup.find(id='mytable')
    trs = table.find_all('tr')  

    col_lst = ['GDAY_DS','T_ID']
    
    for i in trs:
        if '타수' in i.text:
            col_name = i.text.strip().split('\n')[-1]

            if file_name == '투수2.txt': #투수.txt or 투수2.txt
                col_lst += list(map(''.join, zip(*[iter(col_name[:10])]*2)))[1:]
                col_lst += list(map(''.join, zip(*[iter(col_name[10:12])]*1)))
                col_lst.append(col_name[12:14])
                col_lst.append(col_name[14:15])
                col_lst += list(map(''.join, zip(*[iter(col_name[15:])]*2)))[:-1]
                break

            elif file_name == '타자2.txt': #타자.txt or 타자2.txt
                col_lst += list(map(''.join, zip(*[iter(col_name)]*2)))[1:-1]
                break
            
    # dataframe
    data = pd.DataFrame([],columns = col_lst)

    return data

In [11]:
def create_row(data, urls):
    idx = 0
    for url in urls:
        url = url.strip()[1:-1]

        # 날짜
        day_md = url.split('&')[28].split('=')[1].split('-')
        day = '2020{}{}'.format(day_md[0],day_md[1])
        print("=====",day)

        fp = urlopen(url)
        source = fp.read()
        fp.close()

        soup = BeautifulSoup(source)

        table = soup.find(id='mytable')
        trs = table.find_all('tr')
        
        
        for i in range(3,len(trs)):
            tmp = trs[i].text.split()[:len(data.columns)]
            print(tmp)

            for t in team_dict:
                if t in tmp[0]:
                    t_id = team_dict[t]
                    break

            row = [day, t_id]
            value = tmp[2:]

            row += value

            data.loc[idx,:] = row
            idx += 1

    return data

### 2. 함수사용

#### 1) 투수

In [12]:
file_name = '투수2.txt' # file 명 따라 수정
urls, tmp_url = read_txt(file_name)
p_data = make_dataframe(tmp_url, file_name)
pitcher2020 = create_row(p_data, urls)

===== 20200922
['1KT20개', '9.00', '4', '1', '0', '0', '0', '1', '0', '0', '8.0', '8', '8', '36', '31', '10', '0', '0', '2', '5', '2', '0', '5', '130']
['2SK20개', '7.88', '4', '1', '0', '0', '0', '1', '0', '0', '8.0', '7', '7', '35', '30', '7', '1', '0', '0', '5', '0', '0', '5', '149']
['3두산20개', '5.63', '4', '1', '0', '0', '0', '1', '0', '0', '8.0', '5', '5', '35', '29', '6', '0', '0', '1', '4', '0', '1', '16', '145']
['4LG20개', '2.00', '4', '1', '0', '0', '1', '0', '1', '1', '9.0', '2', '2', '32', '31', '3', '0', '0', '1', '1', '0', '0', '9', '141']
['NC20개', '2.00', '4', '1', '0', '0', '1', '0', '0', '1', '9.0', '2', '2', '34', '33', '6', '0', '0', '0', '1', '0', '0', '11', '146']
['6한화20개', '1.00', '4', '1', '0', '0', '1', '0', '2', '0', '9.0', '1', '1', '33', '29', '6', '0', '0', '1', '3', '0', '1', '5', '125']
['KIA20개', '1.00', '4', '1', '0', '0', '0', '1', '0', '0', '9.0', '2', '1', '35', '32', '5', '1', '0', '0', '3', '0', '0', '3', '131']
['8키움20개', '0.00', '3', '1', '0', '0',

In [13]:
pitcher2020

,GDAY_DS,T_ID,출장,선발,완투,완봉,승,패,홀드,세,...,타수,안타,2타,3타,홈런,볼넷,고4,사구,삼진,투구
0,20200922,KT,4,1,0,0,0,1,0,0,...,31,10,0,0,2,5,2,0,5,130
1,20200922,SK,4,1,0,0,0,1,0,0,...,30,7,1,0,0,5,0,0,5,149
2,20200922,OB,4,1,0,0,0,1,0,0,...,29,6,0,0,1,4,0,1,16,145
3,20200922,LG,4,1,0,0,1,0,1,1,...,31,3,0,0,1,1,0,0,9,141
4,20200922,NC,4,1,0,0,1,0,0,1,...,33,6,0,0,0,1,0,0,11,146
5,20200922,HH,4,1,0,0,1,0,2,0,...,29,6,0,0,1,3,0,1,5,125
6,20200922,HT,4,1,0,0,0,1,0,0,...,32,5,1,0,0,3,0,0,3,131
7,20200922,WO,3,1,0,0,1,0,1,1,...,30,4,0,0,0,3,0,0,8,131
8,20200922,LT,3,1,0,0,1,0,0,0,...,28,2,0,0,0,2,0,0,10,122
9,20200922,SS,6,1,0,0,0,1,3,0,...,30,6,1,0,0,4,0,2,4,155


In [14]:
pitcher2020.to_csv("pitcher2020_2.csv",index = False)

#### 2) 타자

In [15]:
file_name = '타자2.txt' # file 명 따라 수정
urls, tmp_url = read_txt(file_name)
h_data = make_dataframe(tmp_url, file_name)
hitter2020 = create_row(h_data, urls)

===== 20200922
['1롯데20개', '.323', '36', '31', '8', '10', '0', '0', '2', '16', '8', '1', '0', '5', '0', '2', '5', '2', '0', '0']
['2LG20개', '.233', '35', '30', '7', '7', '1', '0', '0', '8', '6', '2', '0', '5', '0', '0', '5', '2', '0', '0']
['3두산20개', '.207', '33', '29', '1', '6', '0', '0', '1', '9', '1', '0', '0', '3', '1', '0', '5', '3', '0', '0']
['한화20개', '.207', '35', '29', '5', '6', '0', '0', '1', '9', '5', '0', '0', '4', '1', '0', '16', '0', '1', '0']
['5NC20개', '.200', '36', '30', '3', '6', '1', '0', '0', '7', '2', '1', '0', '4', '2', '0', '4', '1', '0', '0']
['6삼성20개', '.182', '34', '33', '2', '6', '0', '0', '0', '6', '2', '0', '0', '1', '0', '0', '11', '0', '0', '0']
['7키움20개', '.156', '35', '32', '2', '5', '1', '0', '0', '6', '2', '0', '0', '3', '0', '0', '3', '0', '0', '0']
['8KIA20개', '.133', '33', '30', '0', '4', '0', '0', '0', '4', '0', '0', '0', '3', '0', '0', '8', '1', '0', '0']
['9SK20개', '.097', '32', '31', '2', '3', '0', '0', '1', '6', '2', '1', '0', '1', '0', '0', '9

In [16]:
hitter2020

,GDAY_DS,T_ID,타석,타수,득점,안타,2타,3타,홈런,루타,타점,도루,도실,볼넷,사구,고4,삼진,병살,희타,희비
0,20200922,LT,36,31,8,10,0,0,2,16,8,1,0,5,0,2,5,2,0,0
1,20200922,LG,35,30,7,7,1,0,0,8,6,2,0,5,0,0,5,2,0,0
2,20200922,OB,33,29,1,6,0,0,1,9,1,0,0,3,1,0,5,3,0,0
3,20200922,HH,35,29,5,6,0,0,1,9,5,0,0,4,1,0,16,0,1,0
4,20200922,NC,36,30,3,6,1,0,0,7,2,1,0,4,2,0,4,1,0,0
5,20200922,SS,34,33,2,6,0,0,0,6,2,0,0,1,0,0,11,0,0,0
6,20200922,WO,35,32,2,5,1,0,0,6,2,0,0,3,0,0,3,0,0,0
7,20200922,HT,33,30,0,4,0,0,0,4,0,0,0,3,0,0,8,1,0,0
8,20200922,SK,32,31,2,3,0,0,1,6,2,1,0,1,0,0,9,0,0,0
9,20200922,KT,30,28,0,2,0,0,0,2,0,0,1,2,0,0,10,0,0,0


In [17]:
hitter2020.to_csv("hitter2020_2.csv",index = False)

In [18]:
tt = pd.merge(pitcher2020, hitter2020, how = 'left', left_on = ['GDAY_DS','T_ID'], right_on = ['GDAY_DS','T_ID'])
tt[tt["볼넷_y"].isnull()]

,GDAY_DS,T_ID,출장,선발,완투,완봉,승,패,홀드,세,...,타점,도루,도실,볼넷_y,사구_y,고4_y,삼진_y,병살,희타,희비
